<a href="https://colab.research.google.com/github/Parkseojin2001/DeepLearningZeroToAll/blob/main/lab_04_2_load_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Loading Data

- 엄청난 양의 데이터를 한 번에 학습시킬 수 없음

  - 너무 느리다.
  
  - 하드웨어적으로 불가능

**해결책 : Minibatch**

### Minibatch Gradient Descent

- 전체 데이터를 균일하게 나눠서 학습하는 방법

**장점**

- 업데이터를 좀 더 빠르게 할 수 있다.

**단점**

- 전체 데이터를 쓰지 않아서 잘못된 방향으로 업데이트를 할 수 있다.

## PyTorch Dataset

- torch.utils.data.Dataset 상속

- \_\_len\_\_( )

  - 이 데이터셋의 총 데이터 수

- \_\_getitem\_\_()

  - 어떠한 인덱스 idx를 받았을 때, 그에 상응하는 입출력 데이터 반환

In [2]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
  def __init__(self):
    self.x_data = [[73, 80, 75],
                   [93, 88, 93],
                   [89, 91, 90],
                   [96, 98, 100],
                   [73, 66, 70]]
    self.y_data = [[152], [185], [180], [196], [142]]

  def __len__(self):
    return len(self.x_data)

  def __getitem__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = torch.FloatTensor(self.y_data[idx])

    return x, y

dataset = CustomDataset()

## PyTorch DataLoader

- torch.utils.data.DataLoader 사용

- batch_size = 2
  - 각 minibatch의 크기
  - 통상적으로 2의 제곱수로 설정(16, 32, 64,...)
- shuffle = True
  - Epoch 마다 데이터셋을 섞어서, 데이터가 학습되는 순서를 바꾼다.

In [3]:
from torch.utils.data import DataLoader

dataloader = DataLoader(
    dataset,
    batch_size = 2,
    shuffle = True,
)

## Full Code with Dataset and DataLoader

- enumerate(dataloader)
  - minibatch 인덱스와 데이터를 받음

- len(dataloader)
  - 한 epoch당 minibatch 개수

In [7]:
class MultivariateLinearRegression(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(3, 1)

  def forward(self, x):
    return self.linear(x)

In [8]:
model = MultivariateLinearRegression()
optimizer = optim.SGD(model.parameters(), lr = 1e-5)
nb_epochs = 20

for epoch in range(nb_epochs + 1):
  for batch_idx, samples in enumerate(dataloader):
    x_train, y_train = samples
    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(epoch, nb_epochs,
                                                           batch_idx + 1, len(dataloader),
                                                           cost.item()))


Epoch    0/20 Batch 1/3 Cost: 35806.484375
Epoch    0/20 Batch 2/3 Cost: 10742.810547
Epoch    0/20 Batch 3/3 Cost: 4796.435547
Epoch    1/20 Batch 1/3 Cost: 882.092163
Epoch    1/20 Batch 2/3 Cost: 308.734467
Epoch    1/20 Batch 3/3 Cost: 170.444794
Epoch    2/20 Batch 1/3 Cost: 25.166571
Epoch    2/20 Batch 2/3 Cost: 12.290089
Epoch    2/20 Batch 3/3 Cost: 0.127282
Epoch    3/20 Batch 1/3 Cost: 4.204286
Epoch    3/20 Batch 2/3 Cost: 1.055140
Epoch    3/20 Batch 3/3 Cost: 12.507796
Epoch    4/20 Batch 1/3 Cost: 4.350006
Epoch    4/20 Batch 2/3 Cost: 1.849484
Epoch    4/20 Batch 3/3 Cost: 4.611588
Epoch    5/20 Batch 1/3 Cost: 3.016401
Epoch    5/20 Batch 2/3 Cost: 5.475317
Epoch    5/20 Batch 3/3 Cost: 0.945437
Epoch    6/20 Batch 1/3 Cost: 0.428300
Epoch    6/20 Batch 2/3 Cost: 8.958086
Epoch    6/20 Batch 3/3 Cost: 4.670693
Epoch    7/20 Batch 1/3 Cost: 2.997018
Epoch    7/20 Batch 2/3 Cost: 4.776351
Epoch    7/20 Batch 3/3 Cost: 3.287756
Epoch    8/20 Batch 1/3 Cost: 0.154215
Epoch